# AnomalyDetection_1_ExploringData

The first part of the project (separating significant movements from non-significant ones) has been complete, with the following condition having been found:

$$ M =\begin{cases} 
      -1 & g_ig_i > p\left(\frac{1}{\dot{\theta_i}\dot{\theta_i}}\right)\\
      1 & g_ig_i\leq p\left(\frac{1}{\dot{\theta_i}\dot{\theta_i}}\right) \\
   \end{cases}
$$

$$p(x_i) = C_ix_i$$

$$ C_i = \begin{pmatrix}
0.7741697399557282\\
-0.15839741967042406\\
0.09528795099596377\\
-0.004279871380772796
\end{pmatrix} \,\, \mathrm{and} \,\,  x_i = \begin{pmatrix}
x^4\\
x^2\\
x\\
1
\end{pmatrix} $$

On the assumption that this is a good model (ideally given more resources and time, more elaborate testing would have been carried out), the goal now is to find anomalies in time series of the significant movements.

## Libraries

## Data